In [2]:
from elasticsearch import Elasticsearch, exceptions
import json, time
import sys
sys.path.insert(0, '../')
import auth_prop

# time check
start_time = time.time()

# 접속할 elastic 정보
# host = auth_prop.es
host = 'http://localhost:9199'
es = Elasticsearch(host)

# doc_count 초기화
doc_count = 0

# 검색 요청할 인덱스 정의
index = "kpf_bigkindslab_*"

# 검색 요청 쿼리문
# body = {
#     "size":100,
#     "_source": ["content", "providerCd", "newsItemId", "date"],
#     "query": {
#         "bool": {
#             "must": [
#                 {"query_string": {
#                     "default_field": "providerCd",
#                     "query": "*01400551* *01400601* *01400701* *01500051* *01500151* *01500301* *01500401* *01500501* *01500601* *01500701*"
#                 }}
#                 ,{"range": {
#                     "date": {
#                         "gte": "20220101",
#                         "lte": "20220102"
#                     }
#                 }}
#             ]
#         }
#     }
# }
body = {
    "size":100,
    "track_total_hits":"true",
    "sort":[{"@timestamp":{"order":"asc"}}],
    "_source":["content", "providerCd", "newsItemId", "date", "@timestamp"],
    "query":{
        "bool":{
            "must":[
                {"range":{
                    "@timestamp":{
                        "lte":"now-1d/d"
                    }
                }}
            ]
        }
    }
}
# 요청 쿼리문 작성
resp = es.search(index = index, body = body,scroll = '1m')

old_scroll_id = resp['_scroll_id']

res = {}
result = []

print(resp['hits']['hits'][0]['sort'])
# 처음 출력된 결과 저장
for doc in resp['hits']['hits']:
    res['content'] = doc['_source']['content']
    res['newsItemId'] = doc['_source']['newsItemId']
    res['date'] = doc['_source']['date']
    res['providerCd'] = doc['_source']['providerCd']
    result.append(res)

# SCROLL API를 통해 나온 결과 저장
# while len(resp['hits']['hits']):
#     resp = es.scroll(
#         scroll_id = old_scroll_id,
#         scroll = '1m' # length of time to keep search context
#     )
#
#     old_scroll_id = resp['_scroll_id']
#
#     for doc in resp['hits']['hits']:
#         res['content'] = doc['_source']['content']
#         res['newsItemId'] = doc['_source']['newsItemId']
#         res['date'] = doc['_source']['date']
#         res['providerCd'] = doc['_source']['providerCd']
#         result.append(res)
#         print(res['newsItemId'])
#
#     if len(result) == 1000:
#         break

print ("TOTAL TIME:", time.time() - start_time, "seconds.")

C:\Users\user\AppData\Local\Temp\ipykernel_12448\2147705795.py:60: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.search(index = index, body = body,scroll = '1m')


[1671525591478]
TOTAL TIME: 4.6163249015808105 seconds.


In [2]:
print(result[0])
# print(result[0]['newsItemId'])
# print(result[0]['content'])
# print(result[0]['date'])
# print(result[0]['providerCd'])

{'content': '<![CDATA[「신데렐라」신부감을 고르는 왕실무도회에 의붓언니들이 가버린 후에 신데렐라도 무도회에 가고 싶어 울고 있다. 이때 요정 대모가 나타나 온갖 준비를 해주고 12시 전까지 돌아오라고 일러준다.왕자와 즐겁게 춤을 추다가 12시 종소리에 황급히 나오느라고 유리구두 한짝을 잃어버리고 온다. 왕자는 유리구두 한짝을 가지고 나라 안의 모든 처녀에게 신겨 본다. 드디어 신데렐라 집에서 의붓언니들의 욕심 때문에 한바탕 소동이 벌어진 다음에 왕자는 드디어 신데렐라를 알아보게 된다.]]>', 'newsItemId': '01601101.19871107190039009', 'date': '19871107', 'providerCd': '01601101'}


In [20]:
sys.path.insert(0, '../')
import auth_prop
import pymysql

mysql_auth = auth_prop.mysql

# MySQL Connection 연결
con = pymysql.connect(host=mysql_auth['host'], user=mysql_auth['user'], password=mysql_auth['passwd']
                      , db=mysql_auth['db'], charset=mysql_auth['charset'], port=mysql_auth['port'])

# Connection 으로부터 Cursor 생성
cur = con.cursor()

# 업데이트 칠지 ignore 갈지 고민
sql = "INSERT IGNORE INTO ABKL_NEWS_MST(NEWSITEMID, NEWS_CNTS, PUBLISH_DT, PROVIDER_CD) VALUES(%s,%s,%s,%s)"
# sql = """
# INSERT INTO ABKL_NEWS_MST(NEWSITEMID, NEWS_CNTS, PUBLISH_DT, PROVIDER_CD) VALUES(%s,%s,%s,%s)
#     ON DUPLICATE KEY UPDATE NEWS_CNTS = %s, PUBLISH_DT = %S, PROVIDER_CD = %s
# """

cur.execute(sql, (result[0]['newsItemId'], result[0]['content'], result[0]['date'], result[0]['providerCd']))
con.commit()

cur.close()
con.close()

# print(result[0]['content'])
# print(result[0]['newsItemId'])
# print(result[3]['content'])
# print(result[3]['newsItemId'])


for idx,res in enumerate(result):
    if idx == 10:
        break
    print(res)

{'content': '<![CDATA[大學生들의 재치있는 法理論전개 큰 關心『다음과 같이 주문한다. 피고인 王주먹 징역 5년, 마구타(馬毆打) 징역 5년, 모차마(毛差麻) 징역 6월, 그리고 피고인 아민주(阿民主), 선언해(宣言偕)는 각각 무죄를 선고한다.』재판장의 판결선고가 내려지자 2시간 30분동안의 긴 재판과정을때로 박장대소하며 때로는 숨막하는 긴장으로 지켜본 5백여 방청객은 일제히 우뢰와 같은 박수를 보낸다.지난 25일 하오 全北大 합동강당에서 열린 제14회 刑事모의재판 광경이다.法大생들이 연례적으로 개최하는 刑事모의재판은 매년 시사성이 강한 문제들이 집약된 가상적인 범죄를 구성하여 이를 재판에 회부, 현행 법률에 따른 실질적인 재판 과정에 참신하고 명쾌한 법이론 전개, 번뜩이는 재치와 풍자를 조화시킴으로써 대학생의 진면목과 참 멋을 보여주는 행사의 으뜸으로 꼽혀왔다.이날 刑事모의재판의 사건 개요는 다음과 같다.黃土峴시의 不實高敎 교사인 毛差麻·阿民主·宣言偕등이 86년 5월13일 하오 3시께 「師道의 집」에서 敎育民主化를 선언했다. 이에 학교 징계위원회는 이사장 錢慾心의 강력한 요청에 따라 毛교사를 해직시키고 阿·宣 두 교사에게는 3개월의 감봉처분을 내렸다. 세 교사가 이에 반발하며 錢이사장을 찾아가 여학생을 폭행한 일등 각종 비위사실을 폭로하겠다고 위협하자 불안을 느낀 錢은 평소 활동자금을 주어가며 친분을 다져온 폭력해결사 조직인 「잘싸워」파 王주먹·馬毆打에게 돈을 주며 문제해결을 요청하고 이들은 6월5일 하오10시께 시내 暴行동 마자싸 골목에서 야구방망이등을 동원 폭행을 가했다.울분을 느낀 세 교사는 인근 술집에서 술을 마시며 錢이사장에게 강력 항의키로 뜻을 모으고 성격이 급한 毛교사가 錢의 집을 찾아가 강력히 항의하자 평소 고혈압 증세가 있던 錢이 격분 졸도 사망했다.검사는 王과 馬를 폭력행위등 처벌에 관한 법률위반 범죄단체 조직죄로, 毛·阿·宣등 세 교사를 협박죄 살인죄 살인공모죄등을 적용 기소했다.사건개요에 나타난 교육민주화 해결사 성폭행 야

In [ ]:
from elasticsearch import Elasticsearch
import json, time
import sys
sys.path.insert(0, '../')
import auth_prop
from custom_log import Logger
import pymysql
import traceback

# time check
start_time = time.time()

logger = Logger()

newsdb_logger = logger.initLogger()

mysql_auth = auth_prop.mysql

# log DB에서 불러오기
# MySQL Connection 연결
con = pymysql.connect(host=mysql_auth['host'], user=mysql_auth['user'], password=mysql_auth['passwd']
                      , db=mysql_auth['db'], charset=mysql_auth['charset'], port=mysql_auth['port'])

# Connection 으로부터 Cursor 생성
cur = con.cursor()

# 업데이트 칠지 ignore 갈지 고민
sql = "INSERT IGNORE INTO ABKL_NEWS_MST(NEWSITEMID, NEWS_CNTS, PUBLISH_DT, PROVIDER_CD) VALUES(%s,%s,%s,%s)"

# 접속할 elastic 정보
# host = auth_prop.es
host = 'http://localhost:9199'
es = Elasticsearch(host)

# 검색 요청할 인덱스 정의
index = "kpf_bigkindslab_*"
doc_count = 0
total_count = 0
# 검색 요청 쿼리문
init_body = {
    "size":10000,
    "track_total_hits":"true",
    "sort":[{"@timestamp":{"order":"asc"}}],
    "_source":["content", "providerCd", "newsItemId", "date", "@timestamp"],
    "query":{
        "bool":{
            "must":[
                {"range":{
                    "@timestamp":{
                        "lte":"now-1d/d"
                    }
                }},
                {"query_string": {
                    "default_field": "providerCd",
                    "query": "*01100101* *01100201* *01100301* *01100401* *01100501* *01100611* *01100701* *01100801* *01100901* *01101001* *01101101* *01200101* *01200201* *01300101* *01300201* *01400201* *01400351* *01400401* *01400501* *01400551* *01400601* *01400701* *01500051* *01500151* *01500301* *01500401* *01500501* *01500601* *01500701* *01500801* *01500901* *01600201* *01600301* *01600501* *01600801* *01601001* *01601101* *01700101* *01700201* *02100101* *02100201* *02100311* *02100501* *02100601* *02100701* *02100801* *02100851* *07100501* *07101201* *08100101* *08100201* *08100301* *08100401* *08200101*"
                }}
            ]
        }
    }
}

try:

    # 요청 쿼리문 작성
    resp = es.search(index = index, body = init_body)

    # 검색 결과 건수
    total_count = resp['hits']['total']['value']
    sort = 0

    # 처음 출력된 결과 저장
    for doc in resp['hits']['hits']:
        # newsdb_logger.info(doc['_source']['newsItemId'])
        cur.execute(sql, (doc['_source']['newsItemId'], doc['_source']['content'], doc['_source']['date'], doc['_source']['providerCd']))
        con.commit()
        sort = doc['sort'][0]
        doc_count += 1
    newsdb_logger.info(str(doc_count))
    # SCROLL API를 통해 나온 결과 저장
    while len(resp['hits']['hits']):
        body = {
            "size":10000,
            "track_total_hits":"true",
            "sort":[{"@timestamp":{"order":"asc"}}],
            "_source":["content", "providerCd", "newsItemId", "date", "@timestamp"],
            "query":{
                "bool":{
                    "must":[
                        {"range":{
                            "@timestamp":{
                                "lte":"now-1d/d"
                            }
                        }},
                        {"query_string": {
                            "default_field": "providerCd",
                            "query": "*01100101* *01100201* *01100301* *01100401* *01100501* *01100611* *01100701* *01100801* *01100901* *01101001* *01101101* *01200101* *01200201* *01300101* *01300201* *01400201* *01400351* *01400401* *01400501* *01400551* *01400601* *01400701* *01500051* *01500151* *01500301* *01500401* *01500501* *01500601* *01500701* *01500801* *01500901* *01600201* *01600301* *01600501* *01600801* *01601001* *01601101* *01700101* *01700201* *02100101* *02100201* *02100311* *02100501* *02100601* *02100701* *02100801* *02100851* *07100501* *07101201* *08100101* *08100201* *08100301* *08100401* *08200101*"
                          }}
                    ]
                }
            },
            "search_after":[sort]
        }

        resp = es.search(index = index, body = body)
        newsdb_logger.info(str(doc_count))

        for doc in resp['hits']['hits']:
            # newsdb_logger.info(doc['_source']['newsItemId'])
            cur.execute(sql, (doc['_source']['newsItemId'], doc['_source']['content'], doc['_source']['date'], doc['_source']['providerCd']))
            con.commit()
            sort = doc['sort'][0]
            doc_count += 1
except Exception as e: # 에러 처리
    trace_back = traceback.format_exc()
    message = str(e)+ "\n" + str(trace_back)
    newsdb_logger.error('[FAIL] %s', message)
finally:
    cur.close()
    con.close()
    newsdb_logger.debug("total_count : %s", str(total_count))
    newsdb_logger.debug("doc_count : %s", str(doc_count))
    newsdb_logger.debug("sort : %s", str(sort))
    newsdb_logger.debug("TOTAL TIME : %s seconds.", str(time.time() - start_time))

C:\Users\user\AppData\Local\Temp\ipykernel_12448\1088917968.py:65: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.search(index = index, body = init_body)


[INFO    ] <Logger>: 1088917968:78:  10000
[INFO    ] <Logger>: 1088917968:78:  10000
[INFO    ] <Logger>: 1088917968:78:  10000
[INFO    ] <Logger>: 1088917968:78:  10000


C:\Users\user\AppData\Local\Temp\ipykernel_12448\1088917968.py:104: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.search(index = index, body = body)


[INFO    ] <Logger>: 1088917968:105:  10000
[INFO    ] <Logger>: 1088917968:105:  10000
[INFO    ] <Logger>: 1088917968:105:  10000
[INFO    ] <Logger>: 1088917968:105:  10000
[INFO    ] <Logger>: 1088917968:105:  20000
[INFO    ] <Logger>: 1088917968:105:  20000
[INFO    ] <Logger>: 1088917968:105:  20000
[INFO    ] <Logger>: 1088917968:105:  20000
[INFO    ] <Logger>: 1088917968:105:  30000
[INFO    ] <Logger>: 1088917968:105:  30000
[INFO    ] <Logger>: 1088917968:105:  30000
[INFO    ] <Logger>: 1088917968:105:  30000
[INFO    ] <Logger>: 1088917968:105:  40000
[INFO    ] <Logger>: 1088917968:105:  40000
[INFO    ] <Logger>: 1088917968:105:  40000
[INFO    ] <Logger>: 1088917968:105:  40000
[INFO    ] <Logger>: 1088917968:105:  50000
[INFO    ] <Logger>: 1088917968:105:  50000
[INFO    ] <Logger>: 1088917968:105:  50000
[INFO    ] <Logger>: 1088917968:105:  50000
[INFO    ] <Logger>: 1088917968:105:  60000
[INFO    ] <Logger>: 1088917968:105:  60000
[INFO    ] <Logger>: 1088917968:

In [10]:
print(resp['hits']['hits'][0]['sort'][0])

1671525591478


In [10]:
from datetime import datetime

str(datetime.today())[:10]

2023-01-09
